In [216]:
!python3 -m pip install llama_index ChatAnyscale chromadb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
ERROR: Could not find a version that satisfies the requirement ChatAnyscale (from versions: none)
ERROR: No matching distribution found for ChatAnyscale

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [9]:
import gc
from threading import Thread
import torch
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM,
    LlamaTokenizer,
    BitsAndBytesConfig,
    pipeline
)
from transformers import (
    GenerationConfig, 
    TextIteratorStreamer,
    TextStreamer
)
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from langchain import PromptTemplate

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# https://huggingface.co/NousResearch/Nous-Hermes-Llama2-13b
MODEL_ID = "NousResearch/Nous-Hermes-Llama2-13b"
# MODEL_ID = "tiiuae/falcon-40b-instruct"

In [11]:
# quantization config using BnB
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID, 
    load_in_8bit=True, 
    # quantization_config=bnb_config,
    # trust_remote_code=True,
    torch_dtype=torch.float16, 
    device_map="auto"
)

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|██████████| 3/3 [00:16<00:00,  5.40s/it]
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instanc

In [12]:
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID
)
tokenizer.pad_token = tokenizer.eos_token

In [13]:
def generate_alpaca_like_prompt(user_question, user_context):
    """
    Generates a dolly Like prompt for model to respond with context 
    """
    instruction = f"### Instruction:\n{user_question}"
    context = f"### Input:\n{user_context}" if user_context else None
    response = f"### Response:\n"

    prompt = "\n\n".join([i for i in [instruction, context, response] if i is not None])

    return prompt

def tokenize(tokenizer, prompt):
    """ 
    Tokenize your input prompt to provide as an input 
    to the model
    """
    tokenized = tokenizer(prompt, return_tensors="pt")
    input_ids = tokenized.input_ids
    input_ids = input_ids.to(model.device)
    
    return tokenized, input_ids

In [108]:
user_question = "How is UK home office driving down crime in 2023?"

prompt_qna_only = "Answer the question below truthfully, briefly and polietly to the best of your knowledge. If you don't know the answer, say 'I don't know'. Answer in 2 sentences or less."

prompt_rag_only = "You are an assistant for question-answering tasks. You are helpful and friendly. Only answer the question using the additonal context below and if you don't see context, you just say I don't know. Use three sentences maximum, keep the answer concise and try to answer using enumeration."

question_prompt = f"""
{prompt_rag_only}

{user_question}
"""
user_context = None

prompt = generate_alpaca_like_prompt(
    user_question=question_prompt, 
    user_context=user_context
)
print(prompt)

### Instruction:

You are an assistant for question-answering tasks. You are helpful and friendly. Only answer the question using the additonal context below and if you don't see context, you just say I don't know. Use three sentences maximum, keep the answer concise and try to answer using enumeration.

How is UK home office driving down crime in 2023?


### Response:



In [15]:
_, input_ids = tokenize(
    tokenizer=tokenizer, 
    prompt=prompt
)

In [81]:
# Setup the text streamer 
streamer = TextStreamer(tokenizer, skip_prompt=False, skip_special_tokens=True)

In [10]:
with torch.inference_mode():
    outputs = model.generate(
        input_ids=input_ids,
        temperature=0.01,
        top_p=0.95,
        max_new_tokens=512,
        streamer=streamer, 
        do_sample=True,
        use_cache=False
    )
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1417: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


### Instruction:

You are an assistant for question-answering tasks. You are helpful and friendly. Only answer the question using the additonal context below and if you don't see context, you just say I don't know. Use three sentences maximum, keep the answer concise and try to answer using enumeration.

How is UK home office driving down crime in 2023?


### Response:
The UK Home Office is driving down crime in 2023 through a multi-faceted approach that includes increasing police presence, investing in community-based initiatives, and implementing tougher sentencing policies. By focusing on prevention, early intervention, and enforcement, the Home Office aims to reduce crime rates and make communities safer.


## Llama Index

In [67]:
# # Import the prompt wrapper...but for llama index
# from llama_index.prompts.prompts import SimpleInputPrompt
# # Create a system prompt 
# system_prompt = """[INST] <>
# You are a helpful, respectful and honest assistant. Always answer as 
# helpfully as possible, while being safe. Your answers should not include
# any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
# Please ensure that your responses are socially unbiased and positive in nature.

# If a question does not make any sense, or is not factually coherent, explain 
# why instead of answering something not correct. If you don't know the answer 
# to a question, please don't share false information.

# Your goal is to provide answers relating to the financial performance of 
# the company.<>
# """
# # Throw together the query wrapper
# query_wrapper_prompt = SimpleInputPrompt("{query_str} [/INST]")

In [113]:
from langchain import PromptTemplate

# template_old = """
# <s>[INST] <<SYS>>\nYou are an assistant for question-answering tasks. You are helpful and friendly. Only answer the question and do not provide additional information. If you don't know the answer, you just say I don't know. Use three sentences maximum and keep the answer concise.
# <<SYS>>\n
# {context}\n
# {question} [/INST]"""

template = """
### Instruction:

You are an assistant for question-answering tasks. You are helpful and friendly. Only answer the question using the context below and if you don't see context, you just say I don't know. Use three sentences maximum, keep the answer concise and try to answer using enumeration.

{question}

### Input:
{context}

### Response:
"""
prompt_template = PromptTemplate(template=template, input_variables=['context','question'])

In [114]:
print(prompt_template.template)


### Instruction:

You are an assistant for question-answering tasks. You are helpful and friendly. Only answer the question using the context below and if you don't see context, you just say I don't know. Use three sentences maximum, keep the answer concise and try to answer using enumeration.

{question}

### Input:
{context}

### Response:



In [115]:
pipe = pipeline(
    task='text-generation',
    model=model, 
    tokenizer=tokenizer,
    temperature=0.8,
    top_p=0.95,
    max_new_tokens=512,
    do_sample=True,
    use_cache=False,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    streamer=TextStreamer(
        tokenizer, 
        skip_prompt=True, 
        skip_special_tokens=True
    )
)

In [118]:
from langchain.chains import LLMChain
from langchain.llms import HuggingFacePipeline

llm=HuggingFacePipeline(
    pipeline=pipe
)

llm_chain = LLMChain(
    llm=llm, 
    prompt=prompt_template
)

## Doc Download and Split

In [119]:
import chromadb
from langchain.vectorstores import Chroma
import numpy as np
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

In [120]:
loader = PyPDFDirectoryLoader("./documents/bmw/")

In [121]:
documents = loader.load()

In [122]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,
    chunk_overlap=64,
)
docs = text_splitter.split_documents(documents)

In [123]:
avg_doc_length = lambda documents: sum([len(doc.page_content) for doc in documents])//len(documents)
avg_char_count_pre = avg_doc_length(documents)
avg_char_count_post = avg_doc_length(docs)
print(f'Average length among {len(documents)} documents loaded is {avg_char_count_pre} characters.')
print(f'After the split we have {len(docs)} documents more than the original {len(documents)}.')
print(f'Average length among {len(docs)} documents (after split) is {avg_char_count_post} characters.')

Average length among 248 documents loaded is 1687 characters.
After the split we have 548 documents more than the original 248.
Average length among 548 documents (after split) is 788 characters.


## Generate Embeddings

In [124]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
embeddings

HuggingFaceBgeEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
), model_name='BAAI/bge-small-en', cache_folder=None, model_kwargs={'device': 'cuda'}, encode_kwargs={'normalize_embeddings': False}, query_instruction='Represent this question for searching relevant passages: ')

In [125]:
sample_embedding = np.array(embeddings.embed_query(docs[0].page_content))
print("Size of the embedding: ", sample_embedding.shape[0])

Size of the embedding:  384


In [129]:
print(docs[100].page_content)

The LED lights up if the safety function is
switched on.
Safety switch for rear operation
Press the safety switch when transport‐
ing children in the rear; otherwise, injury may
result if the windows are closed without super‐
vision.◀
Roller sunblinds
Roller sunblind for rear window
General information
If you are no longer able to move the roller sun‐
blind for the rear window after having activated
it a number of times in a row, the system is
blocked for a limited time to prevent overheat‐
ing. Let the system cool.
The roller sunblind for the rear window cannot
be moved at low interior temperatures.
Extending or retracting roller blind for
rear window
Press the button.
Roller sunblinds for the rear side
windows
Pull out the roller sunblind at the loop and hook
it onto the bracket.
Do not open the window while the roller
sunblind is raised.
Do not open the window while the roller sun‐
blind is raised; otherwise, there is a risk of
damage at high speeds that may result in per‐
sonal inj

In [130]:
db = Chroma.from_documents(docs, embeddings)

In [4]:
result = db.similarity_search_with_score("What is the recommended tire air pressure?", k=2)

NameError: name 'db' is not defined

In [133]:
print(result[0][0].page_content)

Profile management
Opening the profiles
A different profile can be called up than the
one associated with the remote control cur‐
rently in use.
1."Settings"
2."Profiles"
3.Select a profile.
Called up profile is assigned to the remote
control being used at the time.
Renaming profiles
1."Settings"
2."Profiles"
The current profile is selected.
3.Open "Options".
4."Rename current profile"
Resetting profiles
The settings of the active profile are reset to
their default values.
1."Settings"
2."Profiles"
The current profile is selected.
3.Open "Options".
4."Reset current profile"
Importing profiles
Existing settings and contacts are overwritten
with the imported profile.
1."Settings"
2."Profiles"
3."Import profile"
4.BMW Online: "BMW Online"
USB interface: "USB device"Exporting profiles
Most settings of the active profile and the
saved contacts can be exported.
This can be helpful for securing and retrieving
personal settings, before delivering the vehicle
to a workshop for example.
1."Setti

## Use RetrivalQA Chain to talk to your Documents

In [134]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [135]:
llm_qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=db.as_retriever(
        search_kwargs={"k": 2}
    ),
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": prompt_template
    }
)

In [136]:
query = "What are the safety features of the car?"

In [137]:
response = llm_qa_chain(query)

The safety features of the car include Hill start assistant, Drive-off assistant, Hints, Holder for beverages, Homepage, Hood, Horn, Hotel function, trunk lid, Hot exhaust system, HUD Head-up Display, Hydroplaning, Ignition key, Ignition off, Ignition on, Indication of a flat tire, Indicator and warning lamps, Individual air distribution, Individual settings, Instrument cluster, Instrument cluster, electronic displays, Instrument lighting, Integrated key, Ice warning, Icy roads, Identification marks, tires, Identification number, Important features in the engine compartment, iDrive, Inflation pressure, tires, Inflation pressure warning FTM, Info display, Initialize, Tire Pressure Monitor TPM, Initializing, Flat Tire Monitor FTM, and many more.


In [138]:
query = "What is the approximate weight of the car?"

In [139]:
response = llm_qa_chain(query)

I don't know the approximate weight of the car.


In [140]:
query = "What is Blind Spot Detection?"

In [141]:
response = llm_qa_chain(query)

Active Blind Spot Detection is a system that uses radar sensors to monitor the area behind and next to a vehicle while it is in motion at speeds above 30 mph/50 km/h. The system indicates whether there are vehicles in the blind spot or approaching from behind on the adjacent lane. If the driver attempts to change lanes after setting the turn signal, the system issues a warning if there is a vehicle in the blind spot or approaching from behind. The lamp in the exterior mirror housing lights up dimly, and the steering wheel vibrates to alert the driver. The personal responsibility of the driver is noted, as the system is not a substitute for the driver's personal judgment of the traffic situation.
